In [10]:
import pandas as  pd
import os
from itertools import chain

# Extract from downloaded files into identical csv format

In [ ]:
dir = "Extracted"
sheet = "Page 4 Generator Data"



for filename in os.listdir(dir):
    if filename.endswith('.xlsx'):
        filePath = os.path.join(dir,filename)
        excelFile = pd.read_excel(filePath,sheet_name=sheet,header=None)
       
        row_index = excelFile[excelFile.iloc[:,0]=='Plant Id'].index[0]
        excelFile.columns = excelFile.iloc[row_index]
        excelFile.columns = [col.replace('\n',' ') for col in excelFile.columns]
        excelFile = excelFile.iloc[row_index+1:,:]

        print(filename, row_index)
        excelFile.to_csv('csv/'+filename.removesuffix('.xlsx')+'.csv',index=None)

# Clean up so all files are in the same format

In [83]:
# read in the csv files
dir = "data"

dfs = []
columns = []
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(dir,filename))
        # remove the 'Respondent Frequency' column if it exists
        if 'Respondent Frequency' in df.columns:
            df = df.drop(columns=['Respondent Frequency'])
        # remove the 'Balancing Authority Code' column if it exists
        if 'Balancing Authority Code' in df.columns:
            df = df.drop(columns=['Balancing Authority Code'])
        if 'BA_CODE' in df.columns:
            df = df.drop(columns=['BA_CODE'])
        # Rename the Id columns to ID if it exists
        if 'Plant ID' in df.columns:
            df = df.rename(columns={'Plant ID':'Plant Id',
                                    'Generator ID':'Generator Id',
                                    'Operator ID':'Operator Id',
                                    'Combined Heat & Power Plant':'Combined Heat And Power Plant',
                                    'State':'Plant State',
                                    'EIA Sector Number':'Sector Number',
                                    'Prime Mover Type':'Reported Prime Mover'})
        if 'EIA Sector Number' in df.columns:
            df = df.rename(columns={'EIA Sector Number':'Sector Number'})
        if 'Net Generation Year to Date' in df.columns:
            df = df.rename(columns={'Net Generation Year to Date':'Net Generation Year To Date'})
        if 'Prime Mover Type' in df.columns:
            df = df.rename(columns={'Prime Mover Type':'Reported Prime Mover'})
        if 'State' in df.columns:
            df = df.rename(columns={'State':'Plant State'})
        # resave df to csv
        df.to_csv('data/'+filename,index=None)

        

/var/folders/0m/kh104v054nj9529192f5_0_00000gn/T/ipykernel_97572/192798014.py:8: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,162,163,164,165,166,167,168,169,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(dir,filename))


# Create time series of each generator's output

In [84]:
# read in the csv files
dir = "data"

dfs = []
columns = []
month_dict = {
    'Net Generation January':1,
    'Net Generation February':2,
    'Net Generation March':3,
    'Net Generation April':4,
    'Net Generation May':5,
    'Net Generation June':6,
    'Net Generation July':7,
    'Net Generation August':8,
    'Net Generation September':9,
    'Net Generation October':10,
    'Net Generation November':11,
    'Net Generation December':12
}
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        filePath = os.path.join(dir,filename)
        df = pd.read_csv(filePath)
        # For each net generation column, change the column name to the month and year
        for col in df.columns:
            if 'Net Generation' in col and col in month_dict:
                df = df.rename(columns={col:f'{month_dict[col]}-{df["YEAR"][0]}'})
        # Concatenate the Generator Id and Plant Id columns
        df['Generator Id'] = df['Plant Id'].astype(str) + '-' + df['Generator Id'].astype(str)
        # Use the Generator Id as the index
        df = df.set_index('Generator Id')
        # Drop the Plant Id column
        df = df.drop(columns=['Plant Id', 'Combined Heat And Power Plant', 'Plant Name', 'Operator Name',
        'Plant State', 'Sector Number', 'Reported Prime Mover', 'Plant State', 'YEAR', 'NAICS Code'])


        print(filename, df.shape)
        print(df.columns)
        dfs.append(df)

2020.csv (3844, 17)
Index(['Operator Id', 'Census Region', 'NERC Region', 'Sector Name', '1-2020',
       '2-2020', '3-2020', '4-2020', '5-2020', '6-2020', '7-2020', '8-2020',
       '9-2020', '10-2020', '11-2020', '12-2020',
       'Net Generation Year To Date'],
      dtype='object')
2021.csv (3775, 17)
Index(['Operator Id', 'Census Region', 'NERC Region', 'Sector Name', '1-2021',
       '2-2021', '3-2021', '4-2021', '5-2021', '6-2021', '7-2021', '8-2021',
       '9-2021', '10-2021', '11-2021', '12-2021',
       'Net Generation Year To Date'],
      dtype='object')
2009.csv (2938, 17)
Index(['Operator Id', 'Census Region', 'NERC Region', 'Sector Name', '1-2009',
       '2-2009', '3-2009', '4-2009', '5-2009', '6-2009', '7-2009', '8-2009',
       '9-2009', '10-2009', '11-2009', '12-2009',
       'Net Generation Year To Date'],
      dtype='object')
2022.csv (2754, 17)
Index(['Operator Id', 'Census Region', 'NERC Region', 'Sector Name', '1-2022',
       '2-2022', '3-2022', '4-2022', '5-

/var/folders/0m/kh104v054nj9529192f5_0_00000gn/T/ipykernel_97572/701777706.py:23: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,162,163,164,165,166,167,168,169,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filePath)


KeyError: 'Plant Id'

In [ ]:
# join the dataframes by index
df = pd.concat(dfs, axis=1, join='outer')
df.to_csv('data/combined.csv')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects